### 0. 라이브러리 임포트 및 파일 경로 설정

In [31]:
# Step 0: 라이브러리 임포트 및 파일 경로 설정
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import gc
import os # 파일 존재 여부 확인용

# 훈련 데이터 파일 경로
TRAIN_DATA_PATH = '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/파이널프로젝트 데이터분석_TM/r05_train.parquet'

# 테스트 데이터 파일 경로 (이전 단계에서 저장한 merged_test_df 파일)
TEST_DATA_PATH = '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/파이널프로젝트 데이터분석_TM/r05_test.parquet'

# 제출 파일 저장 경로
SUBMISSION_FILE_PATH = '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/파이널프로젝트 데이터분석_TM/submission_xgb.csv'


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. 데이터 로드

In [33]:
# train 데이터 로드
r05_train = pd.read_parquet(TRAIN_DATA_PATH)

# test 데이터 로드
r05_test = pd.read_parquet(TEST_DATA_PATH)

print(r05_train, r05_test)

         이용금액_R3M_신용체크  이용금액_R3M_신용  _1순위카드이용금액  _2순위카드이용금액    기준년월  \
0                  196          196        3681           0  201807   
1                13475        13475       13323           0  201807   
2                23988        23988       24493           0  201807   
3                 3904         3904        5933           0  201807   
4                 1190            0           0           0  201807   
...                ...          ...         ...         ...     ...   
2399995          10755         7267        5640           0  201812   
2399996          27636        27636       26357           0  201812   
2399997          23187        23187       17171           0  201812   
2399998              0            0           0           0  201812   
2399999          21463        21463        6984       14564  201812   

                   ID Segment  정상청구원금_B0M  정상청구원금_B2M  이용금액_일시불_R12M  ...  \
0        TRAIN_000000       D       14440       16524          20667  

### 2. 데이터 전처리

In [34]:
# 훈련 데이터 준비: ID 컬럼 제거 (나중에 필요 없지만 혹시 모를 상황 대비)
# r05_train에서 ID 컬럼을 제거하고 train_df_processed를 생성합니다.
train_df_processed = r05_train.copy()
train_df_processed = train_df_processed.drop('ID', axis=1)

# 테스트 데이터셋을 위해 원본 ID와 기준년월 저장 (submission 파일용)
# r05_test에서 ID와 기준년월을 가져옵니다.
original_test_id_map = r05_test[['ID', '기준년월']].copy()

# 'Segment' 컬럼은 타겟 변수이므로 전처리 대상 컬럼에서 제외
object_features = [col for col in train_df_processed.columns if train_df_processed[col].dtype == 'object' and col != 'Segment']

# 훈련 데이터의 object 컬럼 원-핫 인코딩
train_df_processed = pd.get_dummies(train_df_processed, columns=object_features, drop_first=False)

# 'Segment' 타겟 변수 인코딩
le = LabelEncoder()
train_df_processed['Category_encoded'] = le.fit_transform(train_df_processed['Segment'])

# 최종 훈련 데이터 특성 컬럼 목록 정의 (타겟 및 인코딩된 타겟 컬럼 제외)
final_train_features = [col for col in train_df_processed.columns if col not in ['Segment', 'Category_encoded']]
print(f"훈련 데이터 전처리 후 최종 특성 수: {len(final_train_features)}")


# 테스트 데이터의 object 컬럼 원-핫 인코딩
# r05_test에서 ID 컬럼 제거 후 test_df_for_processing을 생성합니다.
test_df_for_processing = r05_test.drop('ID', axis=1).copy()
test_df_for_processing = pd.get_dummies(test_df_for_processing, columns=object_features, drop_first=False)

# 훈련 데이터와 테스트 데이터의 컬럼을 일치시킵니다.
# 훈련 데이터에 있는 모든 특성이 테스트 데이터에도 존재하도록 하고, 없으면 0으로 채웁니다.
X_test_aligned = pd.DataFrame(0, index=test_df_for_processing.index, columns=final_train_features)
for col in final_train_features:
    if col in test_df_for_processing.columns:
        X_test_aligned[col] = test_df_for_processing[col]
    # else: 훈련에는 있지만 테스트에는 없는 컬럼은 X_test_aligned에서 0으로 유지됨

# 메모리 최적화: 원본 데이터프레임 r05_train, r05_test는 더 이상 필요 없으므로 삭제
del r05_train, r05_test
gc.collect()

훈련 데이터 전처리 후 최종 특성 수: 56


23

In [35]:
# Step 3: 데이터셋 분리 및 클래스 가중치 계산
print("\nStep 3: 데이터셋 분리 및 클래스 가중치 계산 중...")

# 전처리된 훈련 데이터에서 최종 X와 y 분리
X = train_df_processed[final_train_features]
y = train_df_processed['Category_encoded']

X_test = X_test_aligned # 이전 단계에서 정렬된 테스트 데이터

print(f"X (훈련 데이터 특성) shape: {X.shape}")
print(f"y (훈련 데이터 타겟) shape: {y.shape}")
print(f"X_test (테스트 데이터 특성) shape: {X_test.shape}")

# 클래스 가중치 계산 (불균형 해소)
classes = np.unique(y)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
class_weight_dict = dict(zip(classes, class_weights))
print(f"계산된 클래스 가중치: {class_weight_dict}")
print("클래스 가중치 계산 완료.")

# 메모리 최적화: train_df_processed는 이제 X와 y로 분리되었으므로 삭제
del train_df_processed
gc.collect()


Step 3: 데이터셋 분리 및 클래스 가중치 계산 중...
X (훈련 데이터 특성) shape: (2400000, 56)
y (훈련 데이터 타겟) shape: (2400000,)
X_test (테스트 데이터 특성) shape: (600000, 56)
계산된 클래스 가중치: {np.int64(0): np.float64(493.82716049382714), np.int64(1): np.float64(3333.3333333333335), np.int64(2): np.float64(3.7620503174229953), np.int64(3): np.float64(1.3744051402752246), np.int64(4): np.float64(0.2497330977517778)}
클래스 가중치 계산 완료.


0

In [36]:
# Step 4: XGBoost 모델 정의 및 하이퍼파라미터 튜닝 (RandomizedSearchCV)
print("\nStep 4: XGBoost 모델 정의 및 하이퍼파라미터 튜닝 중 (RandomizedSearchCV)...")

# XGBoost Classifier 정의
xgb_clf = xgb.XGBClassifier(objective='multi:softmax', # 다중 클래스 분류
                            num_class=len(le.classes_), # 타겟 클래스 개수
                            random_state=42,
                            n_jobs=-1, # 모든 CPU 코어 사용
                            tree_method='hist', # 대규모 데이터셋에 효율적인 히스토그램 기반 트리 구축 (GPU 사용 시 'gpu_hist')
                           )

# 하이퍼파라미터 탐색 범위
param_dist = {
    'n_estimators': [100, 200, 300, 500, 700], # 트리의 개수
    'learning_rate': [0.01, 0.05, 0.1, 0.15], # 학습률
    'max_depth': [3, 5, 7, 9], # 트리의 최대 깊이
    'min_child_weight': [1, 3, 5], # 자식 노드에서 필요한 최소 가중치 합
    'gamma': [0, 0.1, 0.2], # 리프 노드에 추가 분할을 할 최소 손실 감소
    'subsample': [0.7, 0.8, 0.9], # 각 부스팅 이터레이션에서 사용할 샘플 비율
    'colsample_bytree': [0.7, 0.8, 0.9], # 각 트리에서 사용할 컬럼 비율
    'lambda': [0.1, 0.5, 1], # L2 정규화
    'alpha': [0.1, 0.5, 1], # L1 정규화
}

# 교차 검증 폴드 설정 (대규모 데이터이므로 3폴드로 줄여 시간 절약)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# RandomizedSearchCV 설정: n_iter는 실제 최적화 시 더 높게 설정하는 것을 권장
random_search = RandomizedSearchCV(estimator=xgb_clf,
                                   param_distributions=param_dist,
                                   n_iter=5, # 테스트를 위해 적은 수의 반복 (시간 절약)
                                   scoring='accuracy',
                                   cv=cv,
                                   verbose=2,
                                   random_state=42, # <-- 이 부분을 'random_state'로 수정했습니다.
                                   n_jobs=-1
                                  )

# 클래스 가중치를 fit 메서드의 sample_weight 인자로 전달
sample_weights = np.array([class_weight_dict[label] for label in y])

# 모델 학습 (RandomizedSearchCV를 통해 최적의 파라미터를 찾음)
random_search.fit(X, y, sample_weight=sample_weights)

best_xgb_model = random_search.best_estimator_
print(f"\n최적의 하이퍼파라미터: {random_search.best_params_}")
print(f"최고 교차 검증 정확도: {random_search.best_score_:.4f}")
print("XGBoost 모델 튜닝 완료.")


Step 4: XGBoost 모델 정의 및 하이퍼파라미터 튜닝 중 (RandomizedSearchCV)...
Fitting 3 folds for each of 5 candidates, totalling 15 fits

최적의 하이퍼파라미터: {'subsample': 0.8, 'n_estimators': 700, 'min_child_weight': 5, 'max_depth': 9, 'learning_rate': 0.15, 'lambda': 0.5, 'gamma': 0.2, 'colsample_bytree': 0.9, 'alpha': 0.5}
최고 교차 검증 정확도: 0.8826
XGBoost 모델 튜닝 완료.


In [37]:
# Step 5: 모델 성능 평가 (훈련 데이터셋에 대한)
print("\nStep 5: 훈련 데이터셋에 대한 모델 성능 평가 중...")

y_pred_train = best_xgb_model.predict(X)
train_accuracy = accuracy_score(y, y_pred_train)
train_classification_report = classification_report(y, y_pred_train, target_names=le.classes_)

print(f"\n훈련 데이터셋 정확도: {train_accuracy:.4f}")
print("훈련 데이터셋 분류 보고서:\n", train_classification_report)
print("훈련 데이터셋 성능 평가 완료.")


Step 5: 훈련 데이터셋에 대한 모델 성능 평가 중...

훈련 데이터셋 정확도: 0.9248
훈련 데이터셋 분류 보고서:
               precision    recall  f1-score   support

           A       0.99      1.00      0.99       972
           B       1.00      1.00      1.00       144
           C       0.81      0.98      0.89    127590
           D       0.71      0.92      0.80    349242
           E       0.99      0.92      0.95   1922052

    accuracy                           0.92   2400000
   macro avg       0.90      0.96      0.93   2400000
weighted avg       0.94      0.92      0.93   2400000

훈련 데이터셋 성능 평가 완료.


In [38]:
# Step 6: 테스트 데이터 예측 및 Submission 파일 생성
print("\nStep 6: 테스트 데이터 예측 및 Submission 파일 생성 중...")

test_predictions_encoded = best_xgb_model.predict(X_test)
test_predictions_decoded = le.inverse_transform(test_predictions_encoded)

# original_test_id_map에는 테스트 데이터의 ID와 기준년월이 포함되어 있습니다.
# 제출 파일에는 ID 컬럼만 필요합니다.
submission_df = pd.DataFrame({
    'ID': original_test_id_map['ID'],
    'Segment': test_predictions_decoded
})

submission_df.to_csv(SUBMISSION_FILE_PATH, index=False)

print(f"\nSubmission 파일이 '{SUBMISSION_FILE_PATH}' 경로에 성공적으로 생성되었습니다.")
print("생성된 submission 파일의 상위 5개 행:")
print(submission_df.head())
print("모든 작업 완료.")

# 최종 메모리 정리
del X, y, X_test, original_test_id_map, best_xgb_model, le, random_search, class_weight_dict
gc.collect()
print("남은 데이터 및 모델 메모리 정리 완료.")


Step 6: 테스트 데이터 예측 및 Submission 파일 생성 중...

Submission 파일이 '/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/파이널프로젝트 데이터분석_TM/submission_xgb.csv' 경로에 성공적으로 생성되었습니다.
생성된 submission 파일의 상위 5개 행:
           ID Segment
0  TEST_00000       E
1  TEST_00001       E
2  TEST_00002       D
3  TEST_00003       E
4  TEST_00004       E
모든 작업 완료.
남은 데이터 및 모델 메모리 정리 완료.


In [45]:
submission = pd.read_csv('/content/drive/MyDrive/3. 수업/01. 멋사 데이터 분석/12_파이널 프로젝트/파이널프로젝트 데이터분석_TM/submission_xgb.csv')
submission.info()
submission['Segment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ID       600000 non-null  object
 1   Segment  600000 non-null  object
dtypes: object(2)
memory usage: 9.2+ MB


,count
Segment,
E,448173
D,112931
C,38749
A,130
B,17
